In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 设置研究区范围
# Altay = ee.Geometry.Rectangle(85,46,94,51.5)
Altay = ee.Geometry.Rectangle(92.5,47.5,93.5,48.5)
Map.addLayer(Altay,{'opacity': 0.5},'Altay')
Map.centerObject(Altay,6) 

roi = ee.Geometry.Point([93.19,48.07])
Map.addLayer(roi,{}, 'point')

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)
def maskL8sr2(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)  
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)


## 指数的计算
def water_index(img):
    image = img
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

In [14]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(Altay) \
    .filterDate('2020-03-01', '2020-12-31')
altay_landsat = landsat_images.first()
print(altay_landsat.getInfo())
Map.addLayer(altay_landsat,visParams,'altay landsat image')
# 使用AWEI进行计算
altay_water = AWEIsh(altay_landsat).select('AWEI').gt(0).selfMask()
Map.addLayer(altay_water,{'palette':['blue']},'altay water')

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7731, 7851], 'crs': 'EPSG:32646', 'crs_transform': [30, 0, 532785, 0, -30, 5531715]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7731, 7851], 'crs': 'EPSG:32646', 'crs_transform': [30, 0, 532785, 0, -30, 5531715]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7731, 7851], 'crs': 'EPSG:32646', 'crs_transform': [30, 0, 532785, 0, -30, 5531715]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7731, 7851], 'crs': 'EPSG:32646', 'crs_transform': [30, 0, 532785, 0, -30, 5531715]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7731, 7851], 'crs': 'EPSG:32646', 'crs_transform': [30, 0, 5327

In [5]:
# # 加载样本集,PearlRiverDeltanew2020_l78,TibetPlateau_new2020_l78,daqing_new2020_l78,tianjin_new2020_l78,wuhan_new2020_l78
samples = ee.FeatureCollection('users/311605001111/wuhan_tibetan/wuhan_2020')
print("初始的样本数： ",samples.size().getInfo())

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

filtered_samples = k_mean(samples)
print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())

# RandomForest预测使用的波段，分类标签
label = 'waterclass'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']

trainedClassifier = ee.Classifier.smileRandomForest(20).train(filtered_samples,label,bands)

初始的样本数：  4952
筛选后，剩余的样本数：  4920


In [15]:
image = water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139026_20200312'))
Map.addLayer(image, visParams,'image')

image_mask2 = maskL8sr2(image)
Map.addLayer(image_mask2, visParams,'image_mask2')

image_mask = maskL8sr(image)
Map.addLayer(image_mask, visParams,'image_mask')


In [16]:
classify_image2 = image_mask2.select(bands).classify(trainedClassifier).eq(1)
Map.addLayer(classify_image2,{'palette':['white','green'],'min':0,'max':1},"classify_image2")

classify_image = image_mask.select(bands).classify(trainedClassifier).eq(1)
Map.addLayer(classify_image,{'palette':['white','green'],'min':0,'max':1},"classify_image")

In [ ]:
water_vector = altay_water.reduceToVectors(scale=30, maxPixels=1000000000)
Map.addLayer(ee.Image().paint(water_vector, 0, 2), {'palette': 'yellow'}, 'water')
# print(water_vector.size().getInfo())

In [ ]:
# 过滤产生特定位置的湖泊
altay_lake = water_vector.filterBounds(roi)
# print(altay_lake.size().getInfo())
Map.addLayer(altay_lake,{},'altay Lake')
# 使用上一步得到的矢量，裁剪影像
altay_lake_grid = altay_water.clip(altay_lake)
Map.addLayer(altay_lake_grid,{'palette':['red']},'altay lake grid')

In [ ]:
# 定义计算面积的函数
def calculate_area(feature):
    return feature.set({'area': feature.geometry().area(1).divide(1e6)})
altay_waterarea = altay_lake.map(calculate_area)

In [ ]:
print(altay_waterarea.first().propertyNames().getInfo())

print(altay_waterarea.first().get('area').getInfo())

areas = altay_waterarea.aggregate_array('area')
print(areas.getInfo())

In [ ]:
# 下载到谷歌硬盘上
prop = ['area','count', 'label']
geemap.ee_export_vector_to_drive(altay_waterarea,description='altay201805',folder = 'altay',file_format='shp',selectors=prop)

In [ ]:
# 下载到本地
proper = ['label']
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'altay.shp')
geemap.ee_export_vector(altay_lake,filename=filename,selectors=proper)

In [ ]:
def calculate_area(feature):
    return feature.set({'area': feature.geometry().area(1).divide(1e6)})
altay_waterarea = water_vector.map(calculate_area).filter(ee.Filter.gte('area',20))
altay_lake = altay_waterarea.filterBounds(roi)
Map.addLayer(altay_lake,{},'altay Lake')

In [ ]:
# 计算筛选后的湖泊面积总和
print(altay_waterarea.aggregate_sum('area').getInfo())

In [ ]:
geemap.ee_export_vector_to_drive(altay_waterarea,description='altay201805',folder = 'altay',file_format='shp',selectors=prop)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

region = [85,46,94,51.5] 
vis = {'bands': ['AWEI'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(altay_water, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[1,1], linestyle=":")
ax.set_title(label = 'altay lake in 2018/05', fontsize=20)